In [20]:
#from google.colab import drive
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [21]:
def tic(name):
    global _start_time
    global _name
    _start_time = time.time()
    _name = name

def tac():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60) 
    print('Tempo de '+_name+' : {}hour:{}min:{}sec'.format(t_hour,t_min,t_sec))

In [22]:
tic('Carregar Arquivos')
#drive.mount('/gdrive')
#%cd /gdrive
#cCaminho='/gdrive/My Drive/Codenation/Desafio_Final/Data/'
cCaminho='Data/'
df_mkt = pd.read_csv(cCaminho+'estaticos_market.csv',index_col = 0)
df_pf1 = pd.read_csv(cCaminho+'estaticos_portfolio1.csv',index_col= 0 )
df_pf2 = pd.read_csv(cCaminho+'estaticos_portfolio2.csv',index_col= 0 )
df_pf2 = pd.merge(df_pf2,df_mkt, on='id', how='inner')
df_pf3 = pd.read_csv(cCaminho+'estaticos_portfolio3.csv',index_col= 0 )
df_pf3 = pd.merge(df_pf3,df_mkt, on='id', how='inner')
tac()

Tempo de Carregar Arquivos : 0hour:0min:9sec


In [23]:
def market_cosine(df, df_pf,columns):
  def fast_clean(df):
    booleandf = df.select_dtypes(include=[bool]).columns
    booleanDictionary = {True: 'T', False: 'F'}
    for column in booleandf:
      df[column] = df[column].map(booleanDictionary)
    df= df.astype(str)
    objectdf  = df.select_dtypes(include=[object]).columns
    for column in objectdf:
      df[column] = df[column].str.replace(' ', '')
      df[column] = np.where(df[column] == 'nan', str(df.index), df[column])
      df[column] = column.replace('_', '')+"_"+df[column]
      df[column] = df[column].str.replace(',', '')
    Maker_Model = list(df.id.unique()) 
    indice = pd.Series(df.index, index=df['id']).drop_duplicates()
    df_id = pd.DataFrame({'id': df['id']})
    df = df.drop(['id'],axis=1)
    df2 = df_join(df) 
    df['id'] = df_id['id']
    df_clean = pd.DataFrame({'clean': df2})
    sent = [row.split(',') for row in  df_clean['clean']]
    return(sent,indice,Maker_Model,df_clean)
  def df_join(df):
    df2 = df.apply(lambda x: ','.join(x.astype(str)), axis=1)
    return(df2)
  print("INICIO MARKET_COSINE") 
  df_mktclean = pd.DataFrame(df, columns=columns)
  df_pfclean = pd.DataFrame(df_pf, columns=columns)
  tic("Limpar Dados")
  sent,indice, Maker_Model, df_clean = fast_clean(df_mktclean)
  tac()
  tic("Tfid")
  tfidf = TfidfVectorizer(sublinear_tf=False,stop_words=None)
  tfidf_matrix = tfidf.fit_transform(df_clean['clean'])
  tac()
  df_clean['matriz'] = list(tfidf.fit_transform(df_clean['clean']).toarray())
  tic("Recomendação")
  requis=df_pf['id']
  result_list=[]
  result_list2=[]
  total = len(requis)
  count = 1
  for element in requis:
    try:
      id = 'id_'+element
      idx = indice[id]
      cosine = cosine_similarity(tfidf_matrix, tfidf_matrix.getrow(idx)) 
      df_final = pd.DataFrame(data=cosine,columns=['cosine'])
      n,o = df_final.shape
      #n = 500
      n_largest =df_final['cosine'].nlargest(n + 1)
      data_day_list = []   
      for item in n_largest.index:
        if n_largest[item]>0.8:
          idx = indice[item:item+1].index[0]
          data_day =(df_mktclean[df_mktclean.id==idx[3:]])
          data_day_list.append(data_day)
      count=count+1
      #clear_output()
      result = pd.concat(data_day_list)
      result_list.append(result)
    except Exception as e:
      print('')
  result_final = pd.concat(result_list)
  result_final.drop_duplicates(keep = 'first', inplace = True)
  result_list2 = []
  result = []
  for element in requis:
      result = (result_final[result_final.id==element]) 
      result_list2.append(result)
  result_final2 = pd.concat(result_list2)
  i1 = len(requis)
  f1,f2 = result_final2.shape
  print("Percentual de item solicitado no resultado: "+str(round(f1/i1*100,2))+'%')
  tac()
  print(" FIM MARKET_COSINE")
  return(result_final)

In [24]:
def market_linear_kernel(df,df_pf,columns):
  def fast_clean(df):
    booleandf = df.select_dtypes(include=[bool]).columns
    booleanDictionary = {True: 'T', False: 'F'}
    for column in booleandf:
      df[column] = df[column].map(booleanDictionary)
    df= df.astype(str)
    objectdf  = df.select_dtypes(include=[object]).columns
    for column in objectdf:
      df[column] = df[column].str.replace(' ', '')
      df[column] = np.where(df[column] == 'nan', str(df.index), df[column])
      df[column] = column.replace('_', '')+"_"+df[column]
      df[column] = df[column].str.replace(',', '')
    Maker_Model = list(df.id.unique()) 
    indice = pd.Series(df.index, index=df['id']).drop_duplicates()
    df_id = pd.DataFrame({'id': df['id']})
    df = df.drop(['id'],axis=1)
    df2 = df_join(df) 
    df['id'] = df_id['id']
    df_clean = pd.DataFrame({'clean': df2})
    sent = [row.split(',') for row in  df_clean['clean']]
    return(sent,indice,Maker_Model,df_clean)
  def df_join(df):
    df2 = df.apply(lambda x: ','.join(x.astype(str)), axis=1)
    return(df2)
  print(" INICIO MARKET_LINEAR_KERNEL")
  df_mktclean = pd.DataFrame(df, columns=columns)
  df_pfclean = pd.DataFrame(df_pf, columns=columns)
  tic("Limpar Dados")
  sent,indice, Maker_Model, df_clean = fast_clean(df_mktclean)
  tac()
  tic("Tfid")
  tfidf = TfidfVectorizer(sublinear_tf=False,stop_words=None)
  tfidf_matrix = tfidf.fit_transform(df_clean['clean'])
  tac()
  df_clean['matriz'] = list(tfidf.fit_transform(df_clean['clean']).toarray())
  tic("Recomendação")
  df_pfclean = pd.DataFrame(df_pf, columns=columns)
  requis=df_pf['id']
  result_list=[]
  result_list2=[]
  total = len(requis)
  count = 1
  for element in requis:
    try:
      id = 'id_'+element
      idx = indice[id]
      #cosine = cosine_similarity(tfidf_matrix, tfidf_matrix.getrow(idx)) # Here I assume that the parent vector is stored as the first row in the dataframe, but you could also store it separately
      cosine = linear_kernel(tfidf_matrix, tfidf_matrix.getrow(idx)) 
      df_final = pd.DataFrame(data=cosine,columns=['cosine'])
      n,o = df_final.shape
      #n = 5000
      n_largest =df_final['cosine'].nlargest(n + 1)
      data_day_list = []   
      for item in n_largest.index:
              if n_largest[item]>0.8:
                idx = indice[item:item+1].index[0]
                data_day =(df[df.id==idx[3:]])
                data_day_list.append(data_day)
      count=count+1
      result = pd.concat(data_day_list)
      result_list.append(result)
    except Exception as e:
      print('')
  result_final = pd.concat(result_list)
  result_final.drop_duplicates(keep = 'first', inplace = True)
  result_list2 = []
  result = []
  for element in requis:
      result = (result_final[result_final.id==element]) 
      result_list2.append(result)
  result_final2 = pd.concat(result_list2)
  i1 = len(requis)
  f1,f2 = result_final2.shape
  print("Percentual de item solicitado no resultado: "+str(round(f1/i1*100,2))+'%')
  tac()
  print(" FIM MARKET_LINEAR_KERNEL")
  return(result_final)

In [44]:
def market_knn(df,df_pf,columns):
  def get_index_from_name(name):
    return df_mktclean[df_mktclean["id"]==name].index.tolist()[0]
  def print_similar_id(query=None):
    data_day_list = []
    found_id = get_index_from_name(query)
    for id in indices[found_id][1:]:
        data_day = (df_mktclean.loc[id]["id"])
        data_day_list.append(data_day)
    return(data_day_list)
  print(" INICIO MARKET_KNN")
  tic("tratamento categorica")
  df_mktclean = pd.DataFrame(df, columns=columns)
  df_pf1clean = pd.DataFrame(df_pf, columns=columns)
  cons = pd.DataFrame({'colunas' : df_mktclean.columns,'tipo' : df_mktclean.dtypes})
  print(cons)
  colunas_categoricas = list(cons[(cons['tipo'] == 'category') & (cons['colunas'] != 'id')]['colunas'])
  columns2 = []
  for coluna in colunas_categoricas:
     df_mktclean[coluna] = df_mktclean[coluna].astype('category')
     df_mktclean[coluna + '_cat'] = df_mktclean[coluna].cat.codes
     df_pf1clean[coluna] = df_pf1clean[coluna].astype('category')
     df_pf1clean[coluna + '_cat'] = df_pf1clean[coluna].cat.codes
     columns2.append(coluna + '_cat')
  print(columns2)
  df_mktclean = pd.DataFrame(df, columns=columns2)
  df_pf1clean = pd.DataFrame(df_pf, columns=columns2)
  X = df_mktclean.iloc[:, 1:3]
  y = df_mktclean.iloc[:, 0]
  Xp = df_pf1clean.iloc[:, 1:3]
  yp = df_pf1clean.iloc[:, 0]
  tic("Minimo e Maximo")
  min_max_scaler = MinMaxScaler()
  market_features = min_max_scaler.fit_transform(X)  
  tac()
  tic("Vizinhos")
  #algorithm{‘auto’, ‘ball_tree’, ‘kd_tree’, ‘brute’}
  #nbrs = NearestNeighbors(n_neighbors=3 ,algorithm='brute', metric='cosine', n_jobs=-1).fit(market_features)
  nbrs = NearestNeighbors(n_neighbors=3 ,algorithm='kd_tree', n_jobs=-1).fit(market_features)
  tac()
  tic("Calculo Distancia")
  distances, indices = nbrs.kneighbors(market_features)
  tac()
  tic("Calculo Recomendação")
  result_list=[]
  data_day_list2 = []
  requis=df_pf['id']
  for element in requis:
    data_day_list = []
    data_day_list = (print_similar_id(query=element))
    for item in data_day_list:
      data_day = (df[df.id==item])
      data_day_list2.append(data_day)
    result = pd.concat(data_day_list2)
    result_list.append(result)
  result_final = pd.concat(result_list)
  result_final.drop_duplicates(keep = 'first', inplace = True)
  tac()
  result_list2 = []
  for element in requis:
      result = (result_final[result_final.id==element]) 
      result_list2.append(result)
  result_final2 = pd.concat(result_list2)
  i1 = len(requis)
  f1,f2 = result_final2.shape
  print("Percentual de item solicitado no resultado: "+str(round(f1/i1*100,2))+'%')
  print(" FIM MARKET_KNN")
  return(result_final)

In [ ]:
columns = ['id','de_ramo','sg_uf','de_natureza_juridica']
result_market_cosine = market_cosine(df_mkt,df_pf3,columns)
result_linear_kernel = market_linear_kernel(df_mkt,df_pf3,columns)
result_market_knn = market_knn(df_mkt,df_pf3,columns)
result_market_cosine.to_csv(cCaminho+'result_market_cosine.csv')
result_linear_kernel.to_csv(cCaminho+'result_linear_kernel.csv')
result_market_knn.to_csv(cCaminho+'market_knn.csv')

In [45]:
result_market_knn = market_knn(df_mkt,df_pf3,columns)

 INICIO MARKET_KNN
                                   colunas      tipo
id                                      id    object
de_ramo                            de_ramo  category
sg_uf                                sg_uf  category
de_natureza_juridica  de_natureza_juridica  category
['de_ramo_cat', 'sg_uf_cat', 'de_natureza_juridica_cat']
Tempo de Minimo e Maximo : 0hour:0min:0sec
Tempo de Vizinhos : 0hour:1min:55sec


KeyboardInterrupt: 